In [1]:
import numpy as np

In [18]:
class Value:
    """ Содержит число и его градиент """

    def __init__(self, data, _children=(), _op=''):
        self.data = data
        self.grad = 0
        
        # Внутренние переменные для построения графа вычислений и подсчёта производных
        self._backward = lambda: None
        self._prev = set(_children)
        self._op = _op # название операции (строка)

    # self * other
    def __mul__(self, other):
        # если умножение происходит с объектом другого типа (не Value), то попытаться сделать из него Value
        other = other if isinstance(other, Value) else Value(other)
        
        # тут задаётся выходное значение операции
        out = Value(self.data * other.data, (self, other), '*')

        # тут задаётся функция передачи градиента через эту операцию
        def _backward():
            
            # подсчёт градиента для текущей переменной
            self.grad += other.data * out.grad
        
            # подсчёт градиента для её множителя
            other.grad += self.data * out.grad
        
        out._backward = _backward
        return out

    # self + other
    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        
        # задайте выходное значение операции
        out = Value(self.data + other.data, (self, other), '+')

        # задайте функцию передачи градиента
        def _backward():
            self.grad += 1 * out.grad
            other.grad += 1 * out.grad
        
        out._backward = _backward
        return out
    
    # self ** other
    def __pow__(self, other):
        assert isinstance(other, (int, float)), "only supporting int/float powers for now"
        
        # задайте выходное значение операции
        out = Value(self.data ** other, (self,), '^')

        # задайте функцию передачи градиента
        def _backward():
            self.grad += other * (self.data ** (other - 1)) * out.grad
        
        out._backward = _backward
        return out
    
    # exp(self)
    def exp(self):
        # задайте выходное значение операции
        out = Value(np.exp(self.data) (self,), 'exp')

        # задайте функцию передачи градиента
        def _backward():
            self.grad = self.data * out.grad
        
        out._backward = _backward
        return out
    
    # sigmoid(self)
    def sigmooid(self):
        # задайте выходное значение операции
        out = Value(1 / (1 + np.exp(-self.data)) (self,), 'sigm')

        # задайте функцию передачи градиента
        def _backward():
            self.grad = out.data * (1 - out.data) * out.grad
        
        out._backward = _backward
        return out

    # Просчитать все градиенты по графу вычислений
    def backward(self):

        # топологическая сортировка всех элементов графа
        # (чтобы все _backward() вызывались в нужном порядке)
        
        topo = []
        visited = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)

        build_topo(self)

        # рассчёт градиентов по графу, начиная с конца
        self.grad = 1
        for v in reversed(topo):
            v._backward()

    
    # Все остальные арифметические операторы задаются через те, что определены выше
    
    def __neg__(self): # -self
        return self * -1

    def __radd__(self, other): # other + self
        other = other if isinstance(other, Value) else Value(other)
        return self + other

    def __sub__(self, other): # self - other
        other = other if isinstance(other, Value) else Value(other)
        return self + (-other)

    def __rmul__(self, other): # other * self
        other = other if isinstance(other, Value) else Value(other)
        return self * other

    def __truediv__(self, other): # self / other
        return self * other**-1

    def __rtruediv__(self, other): # other / self
        return other * self**-1

    # Что выводить в print
    def __repr__(self):
        return f"Value(data={self.data}, grad={self.grad})"

In [39]:
def random_matrix(h, w):
    random_matrix = np.zeros(shape=(h, w), dtype=Value)
    for i in range(h):
        for j in range(w):
            random_matrix[i, j] = Value(np.random.randn())
            
    return random_matrix

In [40]:
a = Value(3)
b = Value(5)

In [41]:
c = a ** 3
c.backward()

In [42]:
a

Value(data=3, grad=27)

In [43]:
random_matrix(3, 5)

array([[Value(data=-1.4573069621263544, grad=0),
        Value(data=1.4918686246953738, grad=0),
        Value(data=1.197962789856867, grad=0),
        Value(data=0.9248548867841742, grad=0),
        Value(data=-1.5443119507064058, grad=0)],
       [Value(data=-1.7489016287741932, grad=0),
        Value(data=-1.3421141302336737, grad=0),
        Value(data=0.05189426224196608, grad=0),
        Value(data=-1.3732968536563033, grad=0),
        Value(data=0.4752142093108078, grad=0)],
       [Value(data=0.5927560143275027, grad=0),
        Value(data=0.4381911981058708, grad=0),
        Value(data=-0.3782364133417441, grad=0),
        Value(data=-0.3238660563486782, grad=0),
        Value(data=-0.9456642547444318, grad=0)]], dtype=object)